In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 1 : data management: get started with Dataflow

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/ml_ops_stage1/get_started_dataflow.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/ml_ops_stage1/get_started_dataflow.ipynb">
      Open in Google Cloud Notebooks
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 1 : data management: get started with Dataflow.

### Dataset

The dataset used for this tutorial is the GSOD dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). The version of the dataset you use only the fields year, month and day to predict the value of mean daily temperature (mean_temp).

### Objective

In this tutorial, you learn how to use `Dataflow` for training with `Vertex AI`.

This tutorial uses the following Google Cloud ML services:

- `Dataflow`
- `BigQuery Datasets`

The steps performed include:

- Offline preprocessing of data:
    - Serially - w/o dataflow
    - Parallel - with dataflow
- Upstream preprocessing of data:
    - tabular data
    - image data

### Recommendations

When doing E2E MLOps on Google Cloud, the following best practices for preprocessing and feeding data during training of custom models:

#### Preprocessing

Data is preprocessed either:

- Offline: The data is preprocessed and stored prior to training.
    - Small datasets: reprocessed and stored when new data.
- Upstream: The data is preprocessed upstream from the model while the data is feed for training.
    - Training on a CPU.
- Downstream: The data is preprocessed downstream in the model while the data is feed for training.
    - Training on a HW accelerator (e.g., GPU/TPU).

#### Model Feeding

Data is feed for model feeding either:

- In-memory: small dataset.
- From disk: large dataset, quick training.
- `Dataflow` from disk: massive dataset, extended training.

#### AutoML

For AutoML training, preprocessing and model feeding are automatically handled.

Alternately for AutoML tabular model training, you can reconfigure the otherwise default preprocessing.

## Installation

Install the latest version of Vertex SDK for Python.

In [ ]:
import os


# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = '--user'
else:
    USER_FLAG = ''

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *Dataflow* library as well.

In [ ]:
! pip3 install -U apache-beam[gcp] $USER_FLAG

Install the latest GA version of *BigQuery* library as well.

In [ ]:
! pip3 install -U google-cloud-bigquery $USER_FLAG

Install the latest GA version of *TensorFlow Data Validation* library as well.

In [ ]:
! pip3 install -U tensorflow-data-validation $USER_FLAG

Install the latest GA version of *TensorFlow Transform* library as well.

In [ ]:
! pip3 install -U tensorflow-transform $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = 'us-central1'  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aip

#### Import Apache Beam

Import the Apache Beam package into your Python environment.

In [ ]:
import apache_beam as beam

#### Import BigQuery

Import the BigQuery package into your Python environment.

In [ ]:
from google.cloud import bigquery

#### Import pandas

Import the pandas package into your Python environment.

In [ ]:
import pandas as pd

#### Import pandas

Import the numpy package into your Python environment.

In [ ]:
import numpy as np

#### Import TFDV

Import the TensorFlow Data Validation (TFDV) package into your Python environment.

In [ ]:
import tensorflow_data_validation as tfdv

#### Import TensorFlow Transform

Import the TensorFlow Transform (TFT) package into your Python environment.

In [ ]:
import tensorflow_transform as tft

### Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, location=REGION)

### Create BigQuery client

Create the BigQuery client.

In [ ]:
bqclient = bigquery.Client()

## Offline preprocessing data with BigQuery table using pandas dataframe

- Offline: The BigQuery table is preprocessed in-memory and stored prior to training.

    - Extract the tabular data into a pandas dataframe.
    - Preprocess the data, per column, within the dataframe.
    - Write the preprocessed dataframe to a new BigQuery table.

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = 'bigquery-public-data.samples.gsod'

### Read the BigQuery dataset into a pandas dataframe

Next, you read a sample of the dataset into a pandas dataframe using BigQuery `list_rows()` and `to_dataframe()` method, as follows:

- `list_rows()`: Performs a query on the specified table and returns a row iterator to the query results. Optionally specify:
 - `selected_fields`: Subset of fields (columns) to return.
 - `max_results`: The maximum number of rows to return. Same as SQL LIMIT command.


- `rows.to_dataframe()`: Invokes the row iterator and reads in the data into a pandas dataframe.

Learn more about [Loading BigQuery table into a dataframe](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string(
    "bigquery-public-data.samples.gsod"
)

rows = bqclient.list_rows(
    table,
    max_results=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ]

)

dataframe = rows.to_dataframe()
print(dataframe.head())

In [ ]:
categories = dataframe['station_number'].unique()

one_hot = pd.get_dummies(categories)
dataframe['station_number'] = one_hot

In [ ]:
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField("station_number", "FLOAT"),  # <-- after one hot encoding
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

NEW_BQ_TABLE = f'{PROJECT_ID}.samples.{DATASET_ALIAS}_transformed'

job = bqclient.load_table_from_dataframe(
    dataframe, NEW_BQ_TABLE, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = bqclient.get_table(NEW_BQ_TABLE)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), NEW_BQ_TABLE
    )
)

## Upstream preprocessing data with tf.data.Dataset generator

### Image data

- Upstream: The data is preprocessed upstream from the model while the data is feed for training.

    - Define preprocessing function:
        - Input: unprocessed batch of tensors
        - Output: preprocessed batch of tensors
    - Use tf.data.Dataset `map()` method to map the preprocessing function to the generator output.

In this example:

- Load CIFAR10 dataset into memory as numpy arrays.
- Create a tf.data.Dataset generator for the in-memory CIFAR10 dataset. *Note*: The pixel data is casted to FLOAT32 to be compatiable with the preprocessing function which outputs the pixel data as FLOAT32.
- Define a preprocessing function to rescale the pixel data by 1/255.0
- Map the preprocessing function to the generator.

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

tf_dataset = tf.data.Dataset.from_tensor_slices((x_train.astype(np.float32), y_train))

print("Before preprocessing")
for batch in tf_dataset:
    print(batch)
    break

def preprocess_fn(inputs, labels):
    inputs /= 255.0
    return tf.cast(inputs, tf.float32), labels

tf_dataset = tf_dataset.map(preprocess_fn)

print("After preprocessing")
for batch in tf_dataset:
    print(batch)
    break

## Upstream preprocessing data with tf.data.Dataset generator

### Tabular data

- Upstream: The data is preprocessed upstream from the model while the data is feed for training.

    - Define preprocessing function:
        - Input: unprocessed batch of tensors
        - Output: preprocessed batch of tensors
    - Use tf.data.Dataset `map()` method to map the preprocessing function to the generator output.

In this example:

- Blah

In [ ]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

tf_dataset = tf.data.Dataset.from_tensor_slices((x_train.astype(np.float32), y_train))

print("Before preprocessing")
for batch in tf_dataset:
    print(batch)
    break

def preprocessing_fn(inputs, labels):
    inputs = tft.scale_to_0_1(inputs)
    return tf.cast(inputs, tf.float32), labels

tf_dataset = tf_dataset.map(preprocessing_fn)

print("After preprocessing")
for batch in tf_dataset:
    print(batch)
    break

## Offline preprocessing with Dataflow

- Generate data chema from BigQuery table.
- Define Beam pipeline to:
    - Split data from BigQuery table into train and eval datasets.
    - Encode datasets as TFRecords, using the data schema.
    - Save the TFRecords as compressed files to Cloud Storage
- Run the Beam pipeline using Dataflow

###  Generate the raw data schema

Generate the data schema on the dataset with the TensorFlow Data Validation (TFDV) package. Use the `infer_schema()` method, with the following parameters:

- `statistics`: The statistics generated by TFDV.

In [ ]:
schema = tfdv.infer_schema(statistics=stats)
print(schema)

In [ ]:
SCHEMA_LOCATION = os.path.join(BUCKET_NAME, "schema.txt")
tfdv.write_schema_text(output_path=SCHEMA_LOCATION, schema=schema)

### Preprocess data with Dataflow

#### Dataset splitting

Next, you preprocess the data using Dataflow. In this example, you query the BigQuery table and split the examples into training and evaluation datasets. For expendiency, the number of examples from the dataset is limited to 500.

In [ ]:
def parse_bq_record(bq_record):
    """Parses a bq_record to a dictionary."""
    output = {}
    for key in bq_record:
        output[key] = [bq_record[key]]
    return output


def split_dataset(bq_row, num_partitions, ratio):
    """Returns a partition number for a given bq_row."""
    import json

    assert num_partitions == len(ratio)
    bucket = sum(map(ord, json.dumps(bq_row))) % sum(ratio)
    total = 0
    for i, part in enumerate(ratio):
        total += part
        if bucket < total:
            return i
    return len(ratio) - 1

def run_pipeline(args):
    """Runs a Beam pipeline to split the dataset"""

    pipeline_options = beam.pipeline.PipelineOptions(flags=[], **args)

    raw_data_query = args["raw_data_query"]
    write_raw_data = args["write_raw_data"]
    exported_data_prefix = args["exported_data_prefix"]
    temp_location = args["temp_location"]
    project = args["project"]

    source_raw_schema = tfdv.load_schema_text(SCHEMA_LOCATION)
    raw_feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(
        source_raw_schema
    ).feature_spec

    raw_metadata = tft.tf_metadata.dataset_metadata.DatasetMetadata(
        tft.tf_metadata.schema_utils.schema_from_feature_spec(raw_feature_spec)
    )

    with beam.Pipeline(options=pipeline_options) as pipeline:
        with tft_beam.Context(temp_location):

            # Read raw BigQuery data.
            raw_train_data, raw_eval_data = (
                pipeline
                | "Read Raw Data"
                >> beam.io.ReadFromBigQuery(
                    query=raw_data_query,
                    project=project,
                    use_standard_sql=True,
                )
                | "Parse Data" >> beam.Map(parse_bq_record)
                | "Split" >> beam.Partition(split_dataset, 2, ratio=[8, 2])
            )

            _ = (
                raw_train_data
                | "Write Raw Train Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(
                        exported_data_prefix, "train/"
                    ),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(schema),
                )
            )

            _ = (
                raw_eval_data
                | "Write Raw Eval Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(
                        exported_data_prefix, "eval/"
                    ),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(schema),
                )
            )

EXPORTED_DATA_PREFIX = os.path.join(BUCKET_NAME, 'exported_data')

QUERY_STRING = "SELECT {0},{1} FROM {2} LIMIT 500".format("station_number,year,month,day", "mean_temp", IMPORT_FILE[5:])
JOB_NAME = "gsod" + TIMESTAMP

args = {
    'runner': 'DirectRunner',
    'raw_data_query': QUERY_STRING,
    'write_raw_data': True,
    'exported_data_prefix': EXPORTED_DATA_PREFIX,
    'temp_location': os.path.join(BUCKET_NAME, 'temp'),
    'project': PROJECT_ID
}

print('Data preprocessing started...')
run_pipeline(args)
print('Data preprocessing completed.')

! gsutil ls $EXPORTED_DATA_PREFIX/train
! gsutil ls $EXPORTED_DATA_PREFIX/eval

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if 'dataset' in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if 'model' in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if 'endpoint' in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if 'dag' in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if 'job' in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if 'batch_predict_job' in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if 'hpt_job' in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if 'BUCKET_NAME' in globals():
        ! gsutil rm -r $BUCKET_NAME